In [1]:
import asyncio
import uuid
from datetime import datetime, timezone
from psql_models import async_session, User, Meeting, Speaker, DiscussionPoint, UserMeeting, DefaultAccess, AccessLevel

async def create_dummy_data():
    async with async_session() as session:
        # Create users
        users = [
            User(
                email=f"user{i}@example.com",
                username=f"user{i}",
                first_name=f"First{i}",
                last_name=f"Last{i}"
            ) for i in range(1, 4)
        ]
        session.add_all(users)
        await session.flush()

        # Create meetings
        meetings = [
            Meeting(
                transcript="Sample transcript for meeting 1",
            ),
            Meeting(
                transcript="Sample transcript for meeting 2",
            )
        ]
        session.add_all(meetings)
        await session.flush()

        # Create speakers
        speakers = [
            Speaker(name="John Doe"),
            Speaker(name="Jane Smith"),
            Speaker(name="Bob Johnson")
        ]
        session.add_all(speakers)
        await session.flush()

        # Create user meetings (sharing permissions)
        user_meetings = [
            UserMeeting(
                meeting_id=meetings[0].meeting_id,
                user_id=users[0].id,
                access_level=AccessLevel.OWNER.value,
                created_by=users[0].id,
                is_owner=True
            ),
            UserMeeting(
                meeting_id=meetings[0].meeting_id,
                user_id=users[1].id,
                access_level=AccessLevel.TRANSCRIPT.value,
                created_by=users[0].id,
                is_owner=False
            )
        ]
        session.add_all(user_meetings)

        # Create discussion points
        discussion_points = [
            DiscussionPoint(
                summary="Key point 1",
                details="Detailed discussion about point 1",
                referenced_text="Original text reference",
                meeting_id=meetings[0].meeting_id,
                speaker_id=speakers[0].id,
                topic_name="Topic 1",
                topic_type="Decision",
                model="gpt-4"
            ),
            DiscussionPoint(
                summary="Key point 2",
                details="Detailed discussion about point 2",
                referenced_text="Original text reference",
                meeting_id=meetings[0].meeting_id,
                speaker_id=speakers[1].id,
                topic_name="Topic 2",
                topic_type="Action Item",
                model="gpt-4"
            )
        ]
        session.add_all(discussion_points)

        # Create default access permissions
        default_access = [
            DefaultAccess(
                owner_user_id=users[0].id,
                granted_user_id=users[1].id,
                access_level=AccessLevel.TRANSCRIPT.value
            )
        ]
        session.add_all(default_access)

        await session.commit()

In [2]:
await create_dummy_data()

In [11]:
from psql_helpers import read_table_async

In [12]:
await read_table_async(DiscussionPoint)

,id,summary_index,summary,details,referenced_text,meeting_id,speaker_id,topic_name,topic_type,model
0,1,NaN,Key point 1,Detailed discussion about point 1,Original text reference,19e2ec78-b310-49a1-818f-00e9fbfcd128,1,Topic 1,Decision,gpt-4
1,2,NaN,Key point 2,Detailed discussion about point 2,Original text reference,19e2ec78-b310-49a1-818f-00e9fbfcd128,2,Topic 2,Action Item,gpt-4
2,3,0.0,The team successfully resolved a technical iss...,"The specific problem was not detailed, but it ...",Ravshan S.K.: Смогли решить проблему. | Dmitr...,7343734f-7311-4844-aa5e-86f1f93595c5,4,Problem resolution,solution,MeetingExtraction
3,4,1.0,Dmitry expressed interest in seeing Ravshan's ...,Dmitry wants to understand how Ravshan approac...,"Dmitry Grankin: Мне интересно посмотреть, как...",7343734f-7311-4844-aa5e-86f1f93595c5,5,Code demonstration,task,MeetingExtraction
4,5,2.0,The team discussed creating a streaming chat a...,Ravshan suggested developing a streaming chat ...,Dmitry Grankin: Давай сделай просто стриминго...,7343734f-7311-4844-aa5e-86f1f93595c5,5,Streaming chat task,task,MeetingExtraction
5,6,3.0,Ravshan needs to set up access to an API for f...,Ravshan mentioned the need for an API key to p...,"Ravshan S.K.: Да у меня нету айпики, надо там...",7343734f-7311-4844-aa5e-86f1f93595c5,4,API access setup,task,MeetingExtraction
6,7,4.0,The team plans to integrate real-time weather ...,Dmitry proposed adding real-time weather acces...,Dmitry Grankin: can't provide real-time infor...,7343734f-7311-4844-aa5e-86f1f93595c5,5,Real-time weather access,task,MeetingExtraction
7,8,5.0,Dmitry discussed the need for hiring developer...,Dmitry expressed that there is no need to hire...,Dmitry Grankin: доступа к погоде короче я теб...,7343734f-7311-4844-aa5e-86f1f93595c5,5,Hiring considerations,concern,MeetingExtraction
8,9,6.0,"Dmitry Grankin is a speaker in the meeting, di...",Dmitry expresses interest in how Ravshan codes...,"Dmitry Grankin: Мне интересно посмотреть, как...",7343734f-7311-4844-aa5e-86f1f93595c5,6,Dmitry Grankin,person,EntityExtraction
9,10,7.0,"Ravshan S.K. is a speaker in the meeting, coll...",Ravshan mentions the need for an API and expre...,"Ravshan S.K.: Да у меня нету айпики, надо там...",7343734f-7311-4844-aa5e-86f1f93595c5,7,Ravshan S.K.,person,EntityExtraction


In [13]:
await read_table_async(UserMeeting)

,id,meeting_id,user_id,access_level,created_at,created_by,is_owner
0,1,19e2ec78-b310-49a1-818f-00e9fbfcd128,fc97506f-0e97-409d-8bdd-699d6dd330a4,owner,2024-10-29 14:16:41.086136+00:00,fc97506f-0e97-409d-8bdd-699d6dd330a4,True
1,2,19e2ec78-b310-49a1-818f-00e9fbfcd128,b15fd48d-e746-41a1-934b-f70cab046c2e,transcript,2024-10-29 14:16:41.086138+00:00,fc97506f-0e97-409d-8bdd-699d6dd330a4,False
2,3,7343734f-7311-4844-aa5e-86f1f93595c5,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,owner,2024-10-29 14:37:42.958347+00:00,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,True


In [14]:
await read_table_async(Speaker)

,id,name
0,1,John Doe
1,2,Jane Smith
2,3,Bob Johnson
3,4,RS
4,5,DG
5,6,Dmitry Grankin
6,7,Ravshan S.K.


In [15]:
await read_table_async(Meeting)

,id,meeting_id,transcript,timestamp
0,1,19e2ec78-b310-49a1-818f-00e9fbfcd128,Sample transcript for meeting 1,2024-10-29 17:16:41.076748
1,2,56481872-8b5c-4420-903f-545c1cfb59af,Sample transcript for meeting 2,2024-10-29 17:16:41.076751
2,3,7343734f-7311-4844-aa5e-86f1f93595c5,"[{'speaker': 'Ravshan S.K.', 'speaker_id': 'TB...",2024-10-25 12:01:02.055999


In [16]:
await read_table_async(DefaultAccess)

,owner_user_id,granted_user_id,access_level,created_at
0,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,8b767b85-44fb-4c20-97df-7aa172242273,search,2024-10-29 14:04:30.543456+00:00
1,fc97506f-0e97-409d-8bdd-699d6dd330a4,b15fd48d-e746-41a1-934b-f70cab046c2e,transcript,2024-10-29 14:16:41.081242+00:00


In [17]:
await read_table_async(User)

,id,email,username,first_name,last_name,image,created_timestamp,updated_timestamp
0,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,2280905@gmail.com,Dmitriy Grankin,Dmitriy,Grankin,https://lh3.googleusercontent.com/a/ACg8ocLu4Z...,2024-10-29 14:02:16.627179+00:00,2024-10-29 14:02:16.627181+00:00
1,8b767b85-44fb-4c20-97df-7aa172242273,olganemirovskaya@gmail.com,Olga Nemirovskaya,Olga,Nemirovskaya,https://lh3.googleusercontent.com/a/ACg8ocKR1i...,2024-10-29 14:02:58.711841+00:00,2024-10-29 14:02:58.711844+00:00
2,fc97506f-0e97-409d-8bdd-699d6dd330a4,user1@example.com,user1,First1,Last1,None,2024-10-29 14:16:41.074831+00:00,2024-10-29 14:16:41.074833+00:00
3,b15fd48d-e746-41a1-934b-f70cab046c2e,user2@example.com,user2,First2,Last2,None,2024-10-29 14:16:41.074836+00:00,2024-10-29 14:16:41.074836+00:00
4,5eaa1e88-0d20-4986-9129-919d1227db52,user3@example.com,user3,First3,Last3,None,2024-10-29 14:16:41.074845+00:00,2024-10-29 14:16:41.074846+00:00


In [ ]:
Speaker